# Analysis "1x8 MEMS Optical Switch"

## Import modules

In [ ]:
# Standard modules
from pathlib import Path

# Dependent packages
import numpy as np
import pandas as pd
import numba
from bokeh.io import output_notebook, output_file, show, save
from bokeh.layouts import column
from bokeh.plotting import figure

## Get data from CSV (THS-3024 output)

In [ ]:
file_name = '~/work/F0000CH1.CSV'
css_path = Path(file_name).expanduser()

data = pd.read_csv(css_path, header=None)
time, volt = data[3].values, data[4].values
del(data)

## FFT the time series data

In [ ]:
eff_volt = volt[178:2428]

N = len(eff_volt)
dt = time[1] - time[0]
eff_time = np.arange(0, N*dt, dt)
freq = np.linspace(0, 1.0/dt, N)
freq_n = freq[:N//2]

f = np.fft.fft(eff_volt)
f = f[:len(f)//2]
amp = np.abs(f)
ph = np.rad2deg(np.arctan2(f.imag, f.real))

## Plot results

In [ ]:
answer = input(
    'Chose 1 or 2!\n'
    '    1. Output notebook\n'
    '    2. Save HTML\n'
)
if answer is '1':
    output_notebook()
elif answer is '2':
    file_name = '1806234355_nosignal.html'
    save_dir = '~/work/mao/plot/'
    file_path = Path(save_dir + file_name).expanduser()
    output_file(filename=file_path)
else:
    raise ValueError('Select 1 or 2.')

### Raw data

In [ ]:
p_raw = figure(title='Raw data', plot_width=800, plot_height=450)
p_raw.line(eff_time, eff_volt, line_color='green', line_width=0.5)

p_raw.title.text_font_size = '24pt'

p_raw.xaxis.axis_label = 'Time[s]'
p_raw.xaxis.axis_label_text_font_size = '16pt'
p_raw.xaxis.axis_label_text_font_style = 'bold'

p_raw.yaxis.axis_label = 'Amplitude[V]'
p_raw.yaxis.axis_label_text_font_size = '16pt'
p_raw.yaxis.axis_label_text_font_style = 'bold'

### FFT (Amplitude)

In [ ]:
# p_amp = figure(title='FFT (Amplitude)', plot_width=800, plot_height=450)
p_amp = figure(title='FFT (Amplitude)', y_axis_type='log', plot_width=800, plot_height=450)
p_amp.line(freq_n, amp, line_color='orange')

p_amp.title.text_font_size = '24pt'

p_amp.xaxis.axis_label = 'Frequency[Hz]'
p_amp.xaxis.axis_label_text_font_size = '16pt'
p_amp.xaxis.axis_label_text_font_style = 'bold'
p_amp.yaxis.axis_label = 'Amplitude[V]'
p_amp.yaxis.axis_label_text_font_size = '16pt'
p_amp.yaxis.axis_label_text_font_style = 'bold'

### FFT (Phase)

In [ ]:
p_ph = figure(title='FFT (Phase)', plot_width=800, plot_height=450)
p_ph.circle(freq_n, ph, size=2, line_color='blue')

p_ph.title.text_font_size = '24pt'

p_ph.xaxis.axis_label = 'Frequency[Hz]'
p_ph.xaxis.axis_label_text_font_size = '16pt'
p_ph.xaxis.axis_label_text_font_style = 'bold'

p_ph.yaxis.axis_label = 'Phase[deg]'
p_ph.yaxis.axis_label_text_font_size = '16pt'
p_ph.yaxis.axis_label_text_font_style = 'bold'

### Show plots

In [ ]:
if answer is '1':
    show(column([p_raw, p_amp, p_ph]))
else:
    save(obj=column([p_raw, p_amp, p_ph]))

## TAV

## Define functions

### Define functions

In [ ]:
@numba.jit(nopython=True)
def calc_tav(data, tau=1.):
    num_data = len(data)
    num_tav = num_data // 2 - 1
    tav = []
    for j in range(1, num_tav):
        sigma = 0
        for i in range(1, num_data-2*j):
            sigma += np.square(data[i+2*j] - 2 * data[i+j] + data[i])
        tav.append(1 / (2 * np.square(tau*j * (num_data-2*j) * sigma)))
        
    return tav

### Calculage TAV

In [ ]:
tav_amp = calc_tav(amp, dt)
tav_ph = calc_tav(ph, dt)

time_lag = np.arange(0, len(tav_amp)* dt, dt)

### Plot TAV

In [ ]:
output_notebook()

### Show plots

In [ ]:
p_tav_amp = figure(title='TAV (Amplitude)', x_axis_type='log', y_axis_type='log', plot_width=800, plot_height=450)
p_tav_amp.line(time_lag, tav_amp, line_color='green')

p_tav_amp.title.text_font_size = '24pt'

p_tav_amp.xaxis.axis_label = 'Time lag[s]'
p_tav_amp.xaxis.axis_label_text_font_size = '16pt'
p_tav_amp.xaxis.axis_label_text_font_style = 'bold'

p_tav_amp.yaxis.axis_label = 'TAV'
p_tav_amp.yaxis.axis_label_text_font_size = '16pt'
p_tav_amp.yaxis.axis_label_text_font_style = 'bold'

In [ ]:
p_tav_ph = figure(title='TAV (Phaes)', x_axis_type='log', y_axis_type='log', plot_width=800, plot_height=450)
p_tav_ph.line(time_lag, tav_amp, line_color='blue')

p_tav_ph.title.text_font_size = '24pt'

p_tav_ph.xaxis.axis_label = 'Time lag[s]'
p_tav_ph.xaxis.axis_label_text_font_size = '16pt'
p_tav_ph.xaxis.axis_label_text_font_style = 'bold'

p_tav_ph.yaxis.axis_label = 'TAV'
p_tav_ph.yaxis.axis_label_text_font_size = '16pt'
p_tav_ph.yaxis.axis_label_text_font_style = 'bold'

In [ ]:
show(column([p_tav_amp, p_tav_ph]))